In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
from pymongo import MongoClient
import re
from datetime import datetime
import copy
import random
from tqdm import tqdm

In [42]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['weibo_crawling']
collection = db['OneAsia_weibo_crawling']
data_list = list(collection.find({"검색키워드": "부산원아시아페스티벌"}, {"_id": 1, "URL": 1}))

In [43]:
data_list1 = copy.deepcopy(data_list)
# 웹드라이버 초기화 및 weibo 검색 페이지 열기
driver = webdriver.Chrome()
driver.maximize_window()

In [44]:
# Weibo 로그인 페이지로 이동
weibo_login_url = 'https://weibo.com/login.php'
driver.get(weibo_login_url)

# 로그인 대기 (로그인 정보 직접 입력)
WebDriverWait(driver, 20).until(EC.url_changes(weibo_login_url))  # 로그인 완료될 때까지 대기 (필요 시 시간 조정)

# 로그인 후 쿠키 저장
cookies = driver.get_cookies()

In [ ]:
def process_url(url):
    try:
        driver.get(url)
        
        # 쿠키 추가
        for cookie in cookies:
            driver.add_cookie(cookie)
        
        # 페이지 새로고침하여 쿠키 적용
        driver.refresh()
        
        # 페이지가 완전히 로드될 때까지 대기
        time.sleep(10) 
        
        # 설명 요소 찾기
        detail_text_element = driver.find_element(By.CLASS_NAME, 'detail_text_1U10O')
        detail_text = detail_text_element.text

        try:
            translate_text_element = driver.find_element(By.CLASS_NAME, 'translate_text_1I4G1')
            translate_text = translate_text_element.text
            filtered_text = detail_text.replace(translate_text, '')
        except NoSuchElementException:
            filtered_text = detail_text  # 번역 텍스트 요소가 없는 경우 원본 텍스트 사용
            
        print(filtered_text)
        
        # MongoDB 업데이트
        collection.update_one({'URL': url}, {'$set': {'설명': filtered_text}})
    
    except Exception as e:
        print(f"URL 처리 중 오류 발생: {url}, 오류 내용: {e}")
        return False
    
    return True

# URL을 하나씩 열기 및 tqdm 사용
for index, data in enumerate(tqdm(data_list, desc="URL 처리 진행 중", unit="URL")):
    url = data['URL']
    success = process_url(url)
    
    if not success:
        # 5분 후 재시도
        print(f"5분 후 재시도: {url}")
        time.sleep(300)
        
        # 2번째 시도 (최대 2번의 시도 후 URL 로드가 불가하다면 다음 URL로 넘어감)
        success = process_url(url)
        
        if not success:
            print(f"{url} 처리 실패, 다음 URL로 넘어갑니다.")
            continue
    
    # 각 요청 사이에 랜덤 대기 시간 추가 (5~10초 사이) → weibo에서 빈번한 접속은 일정 시간의 접속을 차단시킴, 일정 대기 시간을 주어 접속 차단을 방지함
    time.sleep(random.randint(5, 10))

# 드라이버 종료
driver.quit()